In [53]:
import numpy as np

# Exercise 1.1: The way out

In [54]:
M = np.load('maze.npy')

First, we set the diagonal elements of $M$ to zero. $M_{ii}=1$ (as given) would imply that the random walker stays with non-zero probability in the same room within one time step. However, we assume that this does not happen since it would contradict the toy example on the exercise sheet.

In [56]:
for i in range(M.shape[0]):
    M[i,i] = 0

First, we find the shortest way out using the breadth search algorithm.

In [57]:
def breadth_search(starting_point, goal, M):
    '''
    Find the shortest way from starting point to goal in the maze specified by M using the breadth first algorithm.
    '''
    parent_dict = {} # store the parents of each visited node in a dictionary to be able to reconstruct the shortes path
    visited = np.zeros(M.shape[0]) # store if room has been visited (0=False, 1=True)
    visited[starting_point] = 1
    queue = [] # store all rooms that still have to be checked
    queue.append(starting_point)
    counter = 0 # index of next element in queue to be chosen
    while counter <= len(queue): 
        curr_room = queue[counter] # select next element from queue to check
        if curr_room == goal:
            break

        neighbours = np.nonzero(M[curr_room,:]) # find all neighbouring rooms of current room
        for neighbour in neighbours[0]:
            # check if room has already been checked; if this is the case, it can be neglected now
            if visited[neighbour] == 0: 
                queue.append(neighbour) # put room into the queue
                parent_dict[neighbour] = curr_room # save the parent room in the dict
                visited[neighbour] = 1 
        counter += 1
        
    if counter > len(queue):
        return 'There is no way out.'
    
    path = [curr_room]
    # reconstruct the path with help of the dictionary
    while curr_room != starting_point:
        curr_room = parent_dict[curr_room]
        path.append(curr_room)
    
    return len(path)-1, list(reversed(path))

In [58]:
steps, path = breadth_search(0,99,M)
print('minimal number of required steps: ' + str(steps))
print(path)

minimal number of required steps: 13
[0, 1, 4, 78, 77, 80, 83, 84, 85, 89, 91, 94, 96, 99]


The list above displays the sequence of rooms when taking the shortest way out. One has to do 13 steps, i.e. one visits 14 rooms including start and goal.

In order to derive transition probabilites from the matrix $M$, we first have to normalize each row of $M$: For this purpose, we define $T'_{ij}=\frac{M_{ij}}{\sum_{j=0}^{99}M_{ij}}$, where $T'_{ij}$ now denotes the probability to go from room $i$ to room $j$. However, in a transition matrix, it should be the other way round such that $T'_{ij}$ is the probability to go from $j$ to $i$. Thus, we still have to transpose $T'$ to obtain the transition matrix $T=T'^T$.

In [40]:
T = (M.T / np.sum(M, axis=1))

# Expected traversal time

Following the toy example on the exercise sheet, we can write for a maze consisting of $N$ rooms
$$
h_{ij}=1+\sum_{k=1}^{N}P_{ik}h_{kj}\quad\mathrm{with}\quad h_{ii}=0,
$$
where $P$ is the transition matrix and $P_{ij}$ denotes the probability to go from room $i$ to room $j$ in a time step. This equation can be arranged in this way for all $i\neq j$. Rearranging terms provides
$$
h_{ij}-\sum_{k=1}^{N}P_{ik}h_{kj}=1\quad\forall i\neq j
$$
which is a linear system of N-1 equations with N-1 unknown variables ($h_{ij}, i=1...N, i\neq j$). Defining the matrix
$$\bar{P}=\begin{cases}P_{ij}&\mathrm{if}i\neq j\\1&\mathrm{if}i=j\end{cases}$$
and let $\bar{P}_{\setminus j}$ denote the matrix $\bar{P}$ without column and row with index $j$ (thus a (N-1)x(N-1)-matrix), then the system of equations can be rewritten as
$$\bar{P}_{\setminus j}\cdot\vec{h}=\vec{1}$$
where $\vec{h}_i=h_{ij}$ and $\vec{1}$ a (N-1)-dimensional vector of ones.

 <font color="red">Lösen des LGS funktioniert nur, wenn isolierte Räume, die nicht erreicht werden können, vorher aus dem LGS entfernt werden, da die Matrix sonst singular ist. Alternativ: Minimierung des Least Squares Errror</font> 

In [31]:
for i in range(100):
    M[i,i] = 0
T = (M.T / np.maximum(1, np.sum(M, axis=1))).T
for i in range(99):
    T[i,i]=1
    for j in range(99):
        if i!=j:
            T[i,j] = -T[i,j]
T = np.delete(T,(13,32,73,76,90), axis=0)
T = np.delete(T,(13,32,73,76,90), axis=1)
b = np.ones(94)
np.linalg.solve(T[:-1,:-1],b)
#c,_,_,_ = np.linalg.lstsq(T[:-1,:-1],b)

array([  2.20266393e+03,   2.19671785e+03,   2.19671785e+03,
         2.21155610e+03,   2.17193351e+03,   2.24612478e+03,
         2.27265880e+03,   2.10306396e+03,   2.25078225e+03,
         2.27165880e+03,   2.27115880e+03,   2.28803535e+03,
         2.30092288e+03,   2.31181040e+03,   2.31587896e+03,
         2.32324692e+03,   2.32533423e+03,   2.32596791e+03,
         2.33224310e+03,   2.34290176e+03,   2.34159984e+03,
         2.34129387e+03,   2.34050964e+03,   2.35048213e+03,
         2.34835273e+03,   2.35281154e+03,   2.35214096e+03,
         2.35281154e+03,   2.35124684e+03,   2.15563009e+03,
         2.15532239e+03,   2.15339932e+03,   2.15393778e+03,
         2.15266855e+03,   2.14766855e+03,   2.13233521e+03,
         2.16916590e+03,   2.14100188e+03,   2.17089586e+03,
         2.20513012e+03,   2.20699361e+03,   2.20839122e+03,
         2.20818645e+03,   2.20126664e+03,   2.20699361e+03,
         2.20513012e+03,   2.24857778e+03,   2.27008388e+03,
         2.26598266e+03,

In [74]:
def simulate_RW(start, goal, M):
    '''
    Simulate a random walk in maze specified by M with given start and goal.
    The required amount of steps (=time) will be returned.
    '''
    curr_room = start
    time = 0
    while curr_room != goal:
        neighbours = np.nonzero(M[curr_room,:])[0]
        curr_room = np.random.choice(neighbours)
        time += 1
    
    return time

 <font color="red">muss eigentlich für N=100000 durchgeführt werden, dauert bei mir vmtl. eine stunde</font> 

In [103]:
N = 1000
escape_times = np.zeros(N)
for i in range(N):
    escape_times[i] = simulate_RW(0,99,M)
print(np.average(escape_times))

2209.487
